### Módulo Inception

In [2]:
# imports
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras import backend
import numpy as np
import tensorflow as tf

# función para crear un modulo inception sin optimización
def naive_inception_module(layer_in, f1, f2, f3):
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
    # 3x3 conv
    conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
    # 5x5 conv
    conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    # concatenar los filtros, asume que los filtros están en la última dimensión
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out

# definir la entrada del modelo
visible = Input(shape=(256, 256, 3))
# agregar el modulo inception
layer = naive_inception_module(visible, 64, 128, 32)
# crear modelo
model = Model(inputs=visible, outputs=layer)
# resumen
model.summary()

# plot_model(model, show_shapes=True, to_file='naive_inception_module.png')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 256, 256, 64) 256         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 256, 256, 128 3584        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 256, 256, 32) 2432        input_2[0][0]                    
____________________________________________________________________________________________

In [3]:
# Ejemplo de creación de un módulo inception eficiente 
 
# función para crear un módulo inception usando convolución 1x1 para hacerlo más eficiente
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
    # 3x3 conv
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(layer_in)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
    # 5x5 conv
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(layer_in)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
    # concatenate filters, assumes filters/channels last
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out
 
# definir el modelo
visible = Input(shape=(256, 256, 3))
# agregar primer bloque inception
layer = inception_module(visible, 64, 96, 128, 16, 32, 32)
# agregar segundo bloque inception
layer = inception_module(layer, 128, 128, 192, 32, 96, 64)
# crear el modelo
model = Model(inputs=visible, outputs=layer)
# resumen del modelo
model.summary()

#plot_model(model, show_shapes=True, to_file='inception_module.png')

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 256, 256, 96) 384         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 256, 256, 16) 64          input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 256, 256, 3)  0           input_3[0][0]                    
____________________________________________________________________________________________